<a href="https://colab.research.google.com/github/acedesci/scanalytics/blob/master/S8_9_retail_analytics/DT_S9_Module2A_Retail_Price_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this session, we begin by installing the Pyomo package and its utils (GLPK -- GNU Linear Programming Kit). Installing Pyomo and its utils in Colab is straightforward as shown below. If you wish to install them in Anaconda or in a different distribution package, please consult this page http://www.pyomo.org/installation.

In [1]:
# Install Pyomo and GLPK
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

     |████████████████████████████████| 2.4MB 4.9MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 256kB 55.9MB/s 
     |████████████████████████████████| 163kB 55.9MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 134443 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-

# Block 1: Data input

We have been working on this dataset during the past few weeks. The product list below is just simply a subset of those products. In this session, we would like to work on price optimization for the items on that list. In particular, each product can be priced at $2.5, $3.0 or $3.5 and we wish to decide the prices at which these products are offered so that our total revenue is maximized. As we have learned from last week's analysis that the relative price of competing/substitution products is a factor affecting sales, we must optimize the pricing of all these products together to reach the global optimum.

In [0]:
from pyomo.environ import *

productList = ['1600027528', '1600027564', '3000006340', '3800031829']
priceList = [2.5, 3.0, 3.5]
avgPriceValue = 3.0

iIndexList = list(range(len(productList)))
jIndexList = list(range(len(priceList)))

# Block 2: Variable declarations

We use model.x to define the decision variable of our optimization model, which is $x_{ij} \in \{0,1\}$ with $i$ being the product index and $j$ being the price index. Please note that in Python (and many other programming languages), the starting index is 0. By setting $x_{01}=1$, we sell product '1600027528' (product at position 0 on the product list) at price $3.0 (price at position 1 on the price list). Given that the data values for price and predicted sales will be explicitly defined (instantiated) in our model, we call the ConcreteModel class. Please feel free to refer to this page (https://pyomo.readthedocs.io/en/stable/pyomo_overview/abstract_concrete.html) for comparisons between concrete and abstract models.

In [3]:
model = ConcreteModel()
# Variables
model.x = Var(iIndexList, jIndexList, within = Binary)
model.pprint()

3 Set Declarations
    x_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=(0, 3)
        [0, 1, 2, 3]
    x_index_1 : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=(0, 2)
        [0, 1, 2]

1 Var Declarations
    x : Size=12, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True :

# Block 3: Modeling (obj function and constraints)

The general form of the objective function is $\sum_{i=0}^3 \sum_{j=0}^2 p_{ij} \cdot \tilde{D}_{ijk} \cdot x_{ij}$, where $p_{ij}$ is the $j$th price of product $i$, e.g. $p_{i1}$ denotes the price at position 1 on the price list of product $i$. Looking at the price list, we can easily see that $p_{i1}=3.0$. As we defined above, $x_{ij}=1$ when product $i$ is sold at price $j$, 0 otherwise. $\tilde{D}_{ijk}$ is the predicted sales of product $i$ when this product is sold at price $j$ while the average price of all competing products, including product $i$, is equal to $k$. In our optimization model, $k$ is preset to 3.0 (avgPriceValue). By inputting price $p_{ij}$ and average price $k$ into the predictive model we trained and saved last week, we can attain the corresponding predicted sales $\left( \tilde{D}_{ijk} \right)$. We use Objective($\cdot$) to define the objective function and 'sense = maximize' to indicate that the objective is for maximization.

In [0]:
# Objective function

model.OBJ = Objective(sense = maximize, expr = 2.5*95.0*model.x[0,0] + 3.0*67.0*model.x[0,1] + 3.5*46.0*model.x[0,2] 
                      + 2.5*24.0*model.x[1,0] + 3.0*23.0*model.x[1,1] + 3.5*20.0*model.x[1,2]  
                      + 2.5*6.0*model.x[2,0] + 3.0*4.0*model.x[2,1] + 3.5*3.0*model.x[2,2]  
                      + 2.5*33.0*model.x[3,0] + 3.0*24.0*model.x[3,1] + 3.5*20.0*model.x[3,2])

As regards the first set of constraints, we wish to make sure that each product is sold at one price only. Therefore, the general form of this constraint set is $\sum_{j=0}^2 x_{ij} = 1,\forall i\in\{0,1,2,3\}$. We use Constraint($\cdot$) to define the constraints.

In [0]:
# Constraints #1
model.PriceChoiceUPC1 = Constraint(expr = model.x[0,0] + model.x[0,1] + model.x[0,2] == 1)
model.PriceChoiceUPC2 = Constraint(expr = model.x[1,0] + model.x[1,1] + model.x[1,2] == 1)
model.PriceChoiceUPC3 = Constraint(expr = model.x[2,0] + model.x[2,1] + model.x[2,2] == 1)
model.PriceChoiceUPC4 = Constraint(expr = model.x[3,0] + model.x[3,1] + model.x[3,2] == 1)

The second set of constraints ensures that the average price of all the **4** products considered in our optimization model equals the predefined average price, which is $k=\$3.0$ (avgPriceValue) in our model. The general form is

$ \frac{ \sum_{i=0}^3 \sum_{j=0}^2 p_{ij} \cdot x_{ij} }{4} =k \iff \sum_{i=0}^3 \sum_{j=0}^2 p_{ij} \cdot x_{ij} = k\cdot 4$

In [6]:
# Constraints #2

model.sumPrice = Constraint(expr = 2.5*model.x[0,0] + 3.0*model.x[0,1] + 3.5*model.x[0,2] 
                      + 2.5*model.x[1,0] + 3.0*model.x[1,1] + 3.5*model.x[1,2]  
                      + 2.5*model.x[2,0] + 3.0*model.x[2,1] + 3.5*model.x[2,2]  
                      + 2.5*model.x[3,0] + 3.0*model.x[3,1] + 3.5*model.x[3,2] == avgPriceValue*4)

model.pprint()

3 Set Declarations
    x_index : Dim=0, Dimen=2, Size=12, Domain=None, Ordered=False, Bounds=None
        Virtual
    x_index_0 : Dim=0, Dimen=1, Size=4, Domain=None, Ordered=False, Bounds=(0, 3)
        [0, 1, 2, 3]
    x_index_1 : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=(0, 2)
        [0, 1, 2]

1 Var Declarations
    x : Size=12, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :  None :     1 : False :  True : Binary
        (0, 1) :     0 :  None :     1 : False :  True : Binary
        (0, 2) :     0 :  None :     1 : False :  True : Binary
        (1, 0) :     0 :  None :     1 : False :  True : Binary
        (1, 1) :     0 :  None :     1 : False :  True : Binary
        (1, 2) :     0 :  None :     1 : False :  True : Binary
        (2, 0) :     0 :  None :     1 : False :  True : Binary
        (2, 1) :     0 :  None :     1 : False :  True : Binary
        (2, 2) :     0 :  None :     1 : False :  True :

# Block 4: Solution and results

Finally, we call for the solver and obtain the solution. We can see that $x_{00} = x_{12} = x_{22} = x_{30} = 1$ and the optimal objective value is $\$400.5$. In other words, we achieve the optimal revenue of $\$400.5$ when product '1600027528' is sold at price $\$2.5$, products '1600027564' and '3000006340' both  at price $\$3.5$ and product '3800031829' at price $\$2.5$. We can easily double-check that all the constraints are satisfied as shown in the results below.

In [7]:
# Solve the model

opt = SolverFactory('glpk')
opt.solve(model) 

model.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (0, 0) :     0 :   1.0 :     1 : False : False : Binary
        (0, 1) :     0 :   0.0 :     1 : False : False : Binary
        (0, 2) :     0 :   0.0 :     1 : False : False : Binary
        (1, 0) :     0 :   0.0 :     1 : False : False : Binary
        (1, 1) :     0 :   0.0 :     1 : False : False : Binary
        (1, 2) :     0 :   1.0 :     1 : False : False : Binary
        (2, 0) :     0 :   0.0 :     1 : False : False : Binary
        (2, 1) :     0 :   0.0 :     1 : False : False : Binary
        (2, 2) :     0 :   1.0 :     1 : False : False : Binary
        (3, 0) :     0 :   1.0 :     1 : False : False : Binary
        (3, 1) :     0 :   0.0 :     1 : False : False : Binary
        (3, 2) :     0 :   0.0 :     1 : False : False : Binary

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True 